# Географические данные

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Географические-данные" data-toc-modified-id="Географические-данные-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Географические данные</a></span><ul class="toc-item"><li><span><a href="#Как-показать-карту" data-toc-modified-id="Как-показать-карту-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Как показать карту</a></span></li><li><span><a href="#Много-объектов-на-карте" data-toc-modified-id="Много-объектов-на-карте-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Много объектов на карте</a></span></li><li><span><a href="#Собственные-иконки" data-toc-modified-id="Собственные-иконки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Собственные иконки</a></span></li><li><span><a href="#Картограммы-(диаграммы-на-картах)" data-toc-modified-id="Картограммы-(диаграммы-на-картах)-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Картограммы (диаграммы на картах)</a></span></li></ul></li></ul></div>

## Как показать карту

In [1]:
import folium

In [2]:
moscow_lat, moscow_lng = 55.751244, 37.618423

In [3]:
m = folium.Map(location=[moscow_lat, moscow_lng])
display(m)

In [4]:
display(
    folium.Map(
        location=[moscow_lat, moscow_lng],
        zoom_start=16,
        tiles='Stamen Terrain',
    )
)

In [5]:
bolshoi_theatre_lat, bolshoi_theatre_lng = 55.760082, 37.618668

m = folium.Map(
    location=[bolshoi_theatre_lat, bolshoi_theatre_lng], zoom_start=17
)
marker = folium.Marker([bolshoi_theatre_lat, bolshoi_theatre_lng])
marker.add_to(m)
display(m)

In [6]:
place_type = 'Театр'

folium.Marker(
    [55.759700, 37.6207130], tooltip=place_type, popup='Малый театр'
).add_to(m)
display(m)

## Много объектов на карте

In [7]:
import pandas as pd

DATA_HOST_URL = 'https://code.s3.yandex.net'

df = pd.read_csv(
    # shopping centers in Moscow
    DATA_HOST_URL
    + '/datasets/moscow_malls_info.csv'
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      786 non-null    object 
 1   address   786 non-null    object 
 2   hours     685 non-null    object 
 3   lat       786 non-null    float64
 4   lng       786 non-null    float64
 5   rating    786 non-null    float64
 6   district  764 non-null    object 
dtypes: float64(3), object(4)
memory usage: 43.1+ KB


In [8]:
m = folium.Map(location=[55.751244, 37.618423])


def create_marker(row):
    folium.Marker(
        [row['lat'], row['lng']], popup=f"{row['name']} {row['rating']}"
    ).add_to(m)


df.apply(create_marker, axis=1)
m

Здесь могут возникнуть проблемы:

Карта может долго отрисовываться, или рендериться (от англ. render, «выводить, отображать»), на слабых по производительности компьютерах или серверах;
Если на карте много точек, при уменьшении масштаба в ней будет сложно ориентироваться.
Из-за высокой нагрузки карта может зависать при передвижении по ней.
Если отобразить на карте несколько тысяч точек, эти проблемы станут особенно заметны. Помогут кластеры — с их помощью можно поместить на карту много маркеров одновременно.

Отметим все торговые центры на карте с помощью кластеров:

In [9]:
m = folium.Map(location=[moscow_lat, moscow_lng])

import folium.plugins

marker_cluster = folium.plugins.MarkerCluster().add_to(m)


def add_to_cluster(row):
    folium.Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)


df.apply(add_to_cluster, axis=1)
m

## Собственные иконки

In [10]:
m = folium.Map(location=[moscow_lat, moscow_lng])
marker_cluster = folium.plugins.MarkerCluster().add_to(m)

import folium.features
from folium import Marker

icon_url = (
    'https://img.icons8.com/external-wanicon-flat-wanicon/344/'
    'external-mall-shop-and-store-wanicon-flat-wanicon.png'
)


def add_with_icon_to_cluster(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
        # создавать иконку нужно каждый раз зановоdistrict_poly
        icon=folium.features.CustomIcon(icon_url, icon_size=(30, 30)),
    ).add_to(marker_cluster)


df.apply(add_with_icon_to_cluster, axis=1)
m

## Картограммы (диаграммы на картах)

In [11]:
df['district'].unique()

array(['Северный административный округ',
       'Северо-Восточный административный округ', nan,
       'Северо-Западный административный округ',
       'Восточный административный округ',
       'Западный административный округ',
       'Центральный административный округ',
       'Южный административный округ',
       'Юго-Восточный административный округ',
       'Юго-Западный административный округ'], dtype=object)

In [12]:
rating_df = df.groupby('district', as_index=False)['rating'].agg('median')
rating_df

,district,rating
0,Восточный административный округ,4.0
1,Западный административный округ,4.2
2,Северный административный округ,4.2
3,Северо-Восточный административный округ,4.1
4,Северо-Западный административный округ,4.0
5,Центральный административный округ,4.1
6,Юго-Восточный административный округ,4.0
7,Юго-Западный административный округ,4.3
8,Южный административный округ,4.2


Чтобы отобразить округа на карте, folium нужно передать границы каждой области. Для этого можно использовать особый текстовый файл в формате GeoJSON. Файл содержит JSON-описание точек, линий (цепочек точек), полигонов (замкнутых цепочек точек). Каждая точка - это координата в виде "(latitude, longitude)".

Обычно GeoJSON-файлы нужных регионов можно найти в интернете. 

In [13]:
import json
import urllib

district_boundaries = json.loads(
    urllib.request.urlopen(
        'https://code.s3.yandex.net/data-analyst/admin_level_geomap.geojson'
    ).read()
)

In [14]:
# создаём карту Москвы
m = folium.Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
folium.Choropleth(
    geo_data=district_boundaries,
    data=rating_df,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

У конструктора `Choropleth()` много параметров:

- `geo_data` — строка, которая содержит путь к GeoJSON-файлу с информацией о границах регионов. Обратите внимание: читать или обрабатывать файл для построения хороплета не требуется.
- `data` — датафрейм, который, помимо прочего, содержит геоданные — информацию о широте и долготе точек.
- `columns` — список столбцов датафрейма из `data`, которые нужно использовать при построении фоновой картограммы.
- `key_on` — строка, которая содержит название нужного ключа в JSON, например с названием региона. В начале строки указывают `feature.`, а затем название ключа. В GeoJSON-файле с данными Москвы названия округов хранятся в ключе `name`, поэтому достаточно передать параметру `key_on` строку `'feature.name'`. Библиотека найдёт ключ в JSON, достанет соответствующее ему значение и сопоставит его со строками датафрейма `data`.
- `fill_color` — строка, которая задаёт цветовую схему хороплета. Мы использовали схему `YlGn`, которая содержит жёлтые (англ. yellow) и зелёные (англ. green) цвета и их оттенки. Цветовая схема может быть следующей:
![fillcolors_image](https://code.s3.yandex.net/data-analyst/new_images/sphx_glr_colormap_reference_002_2_0x%20(1).webp)
- `fill_opacity` — дробное число от `0` до `1`, которое задаёт прозрачность заливки хороплета. Чем меньше число, тем более прозрачной будет фоновая картограмма.
- `legend_name` — строка с подписью к цветовой легенде графика.